In [1]:
import numpy as np
import scipy.io
import pandas as pd

from statsmodels.stats import multitest
import matplotlib.pyplot as plt
from scipy import stats
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from matplotlib import colors

import neuroCombat

dataset_location = '../corrs/'

In [2]:
# test read in 
corr_file = scipy.io.loadmat(dataset_location + 'correlation_components_0487.mat')
corr = corr_file['corr_components']
corr.shape

(70876, 1)

In [3]:
# read in correlation data 
corr_folder = Path(dataset_location).rglob('*.mat')
files = [x for x in corr_folder]

all_corrs = np.zeros((len(files), corr.shape[0]))
subject_ids = [int(file.name[-8:-4]) for file in files]

for idx, name in enumerate(files):
    corr_file = scipy.io.loadmat(name)
    corr = corr_file['corr_components']
    all_corrs[idx] = np.squeeze(corr)
                    
all_corrs = np.nan_to_num(all_corrs)
all_corrs.shape

(1043, 70876)

In [4]:
participants = pd.read_table('../SRPBS_OPEN/participants.tsv')

participant_data = participants[[int(name[-4:]) in subject_ids for name in participants.participant_id.to_numpy()]]

all_sites = list(np.unique(participants.site.to_numpy()))
participant_label = participant_data.diag.to_numpy() / 2
participant_age = participant_data.age.to_numpy()
participant_sex = participant_data.sex.to_numpy() - 1
participant_hand = participant_data.hand.to_numpy() - 1
participant_site = [all_sites.index(site) for site in participant_data.site.to_numpy()]
participant_protocol = participant_data.protocol.to_numpy()

In [5]:
# dinstinguish between mdd and hc 
controls = all_corrs[participant_label == 0]
patients = all_corrs[participant_label == 1]
controls.shape

(791, 70876)

In [6]:
participant_data[['protocol','age','sex','hand']]

,protocol,age,sex,hand
0,1,23,1,1.0
1,1,23,2,1.0
2,1,26,1,1.0
3,1,23,1,1.0
4,1,24,1,1.0
...,...,...,...,...
1381,14,40,1,1.0
1382,14,31,2,1.0
1383,14,37,1,1.0
1384,14,20,2,1.0


In [12]:
rand_perm = np.random.permutation(all_corrs.shape[0])
train_ids = rand_perm[:800]
test_ids = rand_perm[800:]
train_select = np.zeros(all_corrs.shape[0], dtype=bool)
test_select = np.zeros(all_corrs.shape[0], dtype=bool)
train_select[train_ids] = True
test_select[test_ids] = True

In [13]:
covars = participant_data[['protocol','age','sex','hand']]

harmonised_result = neuroCombat.neuroCombat(all_corrs[train_select, :].T, covars[train_select], batch_col='protocol', categorical_cols=['sex','hand'], continuous_cols=['age'])
protocols_test = np.array(covars[test_select], dtype='object')[:,0] # need this to make it the right format...
harmonised_result_test = neuroCombat.neuroCombatFromTraining(all_corrs[test_select, :].T, protocols_test, harmonised_result['estimates'])

X_train = harmonised_result['data'].T
Y_train = participant_label[train_select]
X_test = harmonised_result_test['data'].T
Y_test = participant_label[test_select]

[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data
[neuroCombatFromTraining] In development ...



### Without harmonisation

In [21]:
clf = LogisticRegression(random_state=1, max_iter=1000).fit(all_corrs[train_select, :], Y_train)
train_prediction = clf.predict(all_corrs[train_select, :])
print(f'train prediction: {100*np.count_nonzero(1-train_prediction)/len(train_prediction)}% 0, {100*np.count_nonzero(train_prediction)/len(train_prediction)}% 1,')
print(f'train accuracy: {clf.score(all_corrs[train_select, :], Y_train)}')
test_prediction = clf.predict(all_corrs[test_select, :])
print(f'test prediction: {100*np.count_nonzero(1-test_prediction)/len(test_prediction)}% 0, {100*np.count_nonzero(test_prediction)/len(test_prediction)}% 1,')
print(f'test accuracy: {clf.score(all_corrs[test_select, :], Y_test)}')

train prediction: 76.125% 0, 23.875% 1,
train accuracy: 1.0
test prediction: 81.89300411522633% 0, 18.106995884773664% 1,
test accuracy: 0.7818930041152263


### With harmonisation

In [22]:
clf = LogisticRegression(random_state=1, max_iter=1000).fit(X_train, Y_train)
train_prediction = clf.predict(X_train)
print(f'train prediction: {100*np.count_nonzero(1-train_prediction)/len(train_prediction)}% 0, {100*np.count_nonzero(train_prediction)/len(train_prediction)}% 1,')
print(f'train accuracy: {clf.score(X_train, Y_train)}')
test_prediction = clf.predict(X_test)
print(f'test prediction: {100*np.count_nonzero(1-test_prediction)/len(test_prediction)}% 0, {100*np.count_nonzero(test_prediction)/len(test_prediction)}% 1,')
print(f'test accuracy: {clf.score(X_test, Y_test)}')

train prediction: 76.125% 0, 23.875% 1,
train accuracy: 1.0
test prediction: 86.0082304526749% 0, 13.991769547325102% 1,
test accuracy: 0.7407407407407407
